# Capstone project Week 3

In [279]:
import requests
import pandas as pd

#### Let's get the data using requests package and `get` http call

In [283]:
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [ ]:
#### I am going to use lxml for parsing the html

In [190]:
import lxml.html as lh
xml = lh.fromstring(response.content)

In [ ]:
#### I am going to find all tables on that wiki page and then select the first table.
#### Assumption here is that first table is always the data. 

In [191]:
table = xml.findall(".//*/table")

In [280]:
#### Get all trs from the table
#### which has header and rows
#### we then get contents from all those rows and create a list of lists

In [284]:
header, *rows = table[0].xpath("tbody/tr")
hdr = []
for h in header.xpath("th"):
    hdr.append(h.text_content().strip())

newrows = []
    
for row in rows:
    x = []
    for column in row.xpath("td"):
        x.append(column.text_content().strip())
    newrows.append(x)
 


In [281]:
#### we then create a dataframe from the result above and process it as requested.
#### 1. We filter out all the rows those has Borough is 'Not assigned'
#### 2. Then those with Neighbourhood not assigned we put Borough as Neighbourhood
#### 3. Then we group by Postcode and concatenate all Neighbourhoods (comma separated)

In [245]:
dataframe = pd.DataFrame(newrows)
dataframe.columns = hdr
dataframe.head()
filtered  = dataframe[dataframe.Borough != "Not assigned"]
def handle_non_assigned(row):
#    print(row)
    if(row.Neighbourhood == "Not assigned"):
        row["Neighbourhood"] = row["Borough"]
        return row
    else:
        return row
    
filtered = filtered.apply(handle_non_assigned, axis=1)
filtered
x = filtered.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x: ",".join(x)).reset_index()


In [ ]:
#### We get 103 rows in total

In [250]:
x.shape

(103, 3)